# 특성 선택을 사용한 차원 축소

- 특성 선택: 고품질의 정보가 많은 특성은 선택하고 덜 유요한 특성은 버리는 방식
- 필터: 통계적인 속성을 조사하여 가장 뛰어난 특성을 선택
- 래퍼: 시행착오를 통해 가장 높은 품질의 예측을 만드는 특성의 부분 조합을 찾아 선택
- 임베디드: 학습 알고리즘 훈련 단계를 확장, 혹은 일부로 구성하여 좋은 특성의 부분 조합 선택

### 분산을 기준으로 수치 특성 선택

- 분산 기준 설정(VT)은ㅇ 가장 기본적인 특성 선택 방법
- 분산이 높은 특성보다 분산이 낮은 특성이 효과적이지 않다는 것에 기반
- x는 특성 벡터이고 xi는 개별 특성값, u 는 특성의 평균값
- VT는 분산의 기준값을 수동으로 선택하기 때문에 자가 판단해야함
- 분산은 특성의 제곱 단위로 특성의 단위가 다르면 VT가 동작하지 않음


In [1]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris() 
features = iris.data 
target = iris.target

#기준값 생성
thresholder = VarianceThreshold(threshold=.5)
#기준값보다 높은특성 선택
features_high_variance = thresholder.fit_transform(features)

#선택한 특성 확인
features_high_variance[0:3]
#분산 확인
thresholder.variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [2]:
#특성이(평균이 0,단위 분산으로)표준화 되어 있으면 분산 기준은 동작 않함

from sklearn.preprocessing import StandardScaler

#특성 행렬을 표준화함
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

#각 특성의 분산을 계산
selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

### 분산을 기준으로 이진 특성 선택

- 이진 범주형 특성에서 베르누이 확률 변수의 분산이 기준값 이상인 특성을 선택할 수 있도록 분산이 낮은 특성(정보가 적은 특성) 삭제
- p는 클래스 1의 샘플 비율
- p값을 설정 하여 샘플의 대다수가 한 개의 클래스에 속한 특성 삭제 가능
- varainceThreshold 클래스는 수치 특성, 이진 특성에 상관없이 넘파이 var 함수를 사용하여 분산 계산
- threshold 매개변수의 기본값은 0으로 모든 특성 선택
- var 함수로 이진특성의 분산을 계산하면 베르누이 확률 변수의 분산 p(1-p)와 동일

In [3]:
from sklearn.feature_selection import VarianceThreshold

features = [[0, 1, 0], # 예제 특성 행렬
            [0, 1, 1], # 특성 0: 80%가 클래스 0
            [0, 1, 0], # 특성 1: 80%가 클래스 1
            [0, 1, 1], # 특성 2: 60%가 클래스 0, 40%는 클래스 1
            [1, 0, 0]]

# 분산을 기준으로 선택
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)
thresholder.variances_

import numpy as np

#numpy var 함수를 사용하여 분산 계산
np.var(features, axis=0)

array([0.16, 0.16, 0.24])

### 상관관계가 큰 특성 삭제

- 특성 행렬에서 상관관계 행렬을 사용하여 상관관계가 큰 특성 확인, 하나를 삭제
- 두 가지 특성의 상관관계가 크다면, 중복된 특성을 포함하는 것과 같음
1) 모든 특성에 대한 상관관계 행렬 생성
2) 상관관계 행렬의 상삼각 행렬을 살펴서 크게 상관된 특성의 쌍을 확인함
3) 특성 행렬에서 이런 특성 중 하나 삭제

- 상관관계 행렬은 numpy corrcoef()로 구함(특성이 행에 있을것으로 기대)
- 특성이 열에 있다면 rowvar 매개변수를 False로 지정
- np.triu()는 배열에서 상삼각 행렬을 추출하여 반환
- 매개변수 k가 기본값 0이 되면 반환되는 행렬에 대각원소가 포함
- k값이 커질수록 대각 원소에서 k만큼 떨어진 원소부터 포함
- np.tril()는 주어진 배열에서 하삼각 행렬을 추출 반환


In [4]:
import pandas as pd
import numpy as np

# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1], [2, 2, 0], [3, 3, 1], [4, 4, 0], [5, 5, 1],
                     [6, 6, 0], [7, 7, 1], [8, 7, 0], [9, 7, 1]])

#특성 행렬을 DataFrame으로 변환
dataframe = pd.DataFrame(features)
#상관관계 행렬 생성
corr_matrix = dataframe.corr().abs()

# 상관관계 행렬의 상삼각(upper triangle) 행렬 선택
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# 상관 계수가 0.95보다 큰 특성 열의 인덱스 탐색.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [5]:
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3) 
#상관관계 행렬
dataframe.corr()
#상관관계 행렬의 상삼각 행렬
upper

np.corrcoef(features, rowvar=False)

np.triu(np.ones((4, 4)), k=2)
np.tril(np.ones((4, 4)), k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

### 분류 작업에 관련 없는 특성 삭제

- 범주형 타깃 벡터에서 관련 없는 특성을 삭제하기 위해, 타깃 벡터 사이의 카이제곱 통계를 계산
- 카이제곱 통계는 두범주형 벡터의 독립성 평가
- 카이제곱 통게는 범주형 특성의 각 클래스별 샘플 빈도와 이 특성이 타깃 벡터와 독립적이라면 기대할 수 있는 값의 차이
- 카이제곱 톡성은 관찰 빈도와 관계가 없다고 기대하는 빈도 사이에 얼마나 차이가 있는지 알려주는 하나의 숫자
- 특성과 타깃 벡터 사이의 카이제곱 통계를 계산하면 둘 사이의 독립성 측정 가능

- 특성이 수치형 특성이라면 f_classif 사용, 각 특성과 타깃 벡터 사이에 분산 분석(ANOVA)와 F-값 통계 계산 가능
- F-값 점수는 타깃 벡터로 수치형 특성을 그룹핑하여 각 그룹의 평균이 크게 차이 나는지 평가

In [7]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

iris = load_iris()
features = iris.data
target = iris.target
# 범주형 데이터를 정수형으로 변환
features = features.astype(int)
 
#카이제곱 통계값이 가장 큰 특성 두 개를 선택
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [8]:
# F-값이 가장 높은 특성 두 개 선택
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

# 특정 특성 개수를 선택하는 대신 Selectpercentile를 사용하여 특성의 상위 n 퍼센트 선택 가능
from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성 선택
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2
원본 특성 개수: 4
줄어든 특성 개수: 3


In [10]:
#전체 평균과 클래스 평균을 계산
total_mean = np.mean(features, axis=0)
total_mean

class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

#ss_total 계산
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between

ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total

#ss_beteen과 ss_tatal을 F-값 공식에 대입
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f

#F-값 scores_속성에서 확인
fvalue_selector.scores_ 

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

### 재귀적 특성 제거(recursive feature elimination)

- 사이킷런의 FRECV를 사용하여 재귀적 특성 제거를 교차 검증으로 수행 가능
- 모델 성능이 나빠질 때까지 특성을 제거하면서 반복적 모델 훈련

- 교차검증(CV)사용하여 RFE 과정에서 남길 특성의 최적 갯수 탐색 가능
- 구체적으로 매 반복 후에 CV를 사용한 RFE에서 교차검증을 사용하여 모델 평가
- 특성을 제거한 후 모델의 CV결과가 향상되었다면 반복으로 계속 진행
- 어떤 특성을 제거한 후에 모델의 CV결과가 더 나빠지면 삭제한 특성을 다시 복원하고 이특성 조합을 최선으로 선택함
- 사이킷런의 RFECV는 CV를 사용한 REF구현으로 estimator 매개 변수에는 훈련한 모델의 객체 전달
- step 매개변수는 매반복에서 삭제할 특성의 갯수나 비율 정함
- scoring 매개변수에는 교차검증 동안 사용할 모델의 평가 지표 설정

In [11]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성
features, target = make_regression(n_samples = 10000,n_features = 100,
                                   n_informative = 2,random_state = 1)

# 선형 회귀 모델 생성
ols = linear_model.LinearRegression()

# 재귀적으로 특성 제거
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799,  0.7031277 ],
       [-1.07500204,  2.56148527],
       [ 1.37940721, -1.77039484],
       ...,
       [-0.80331656, -1.60648007],
       [ 0.39508844, -1.34564911],
       [-0.55383035,  0.82880112]])

In [15]:
rfecv.n_features_ # 최선의 특성 개수
rfecv.support_ # 선택된 특성이 표시된 불리언 마스크
rfecv.ranking_ # 특성의 순위: 최고(1)에서 최악(96)까지

from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=3)
rfe.fit(features, target)
rfe.transform(features)

np.all(rfe.support_ == rfecv.support_)

False